In [1]:
! pip install pyodbc
! pip install kafka-python==2.0.2
!pip install pymssql==2.2.5

You should consider upgrading via the '/data/jupyter-lab/provas-env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/jupyter-lab/provas-env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/jupyter-lab/provas-env/bin/python -m pip install --upgrade pip' command.


In [2]:
import json
from kafka import KafkaConsumer
import pymssql
from datetime import datetime, timedelta


In [4]:
kafka_topic = "asm-sql-data"
consumer = KafkaConsumer(kafka_topic, bootstrap_servers=["10.210.4.117:9092","10.210.4.118:9092","10.210.4.119:9092"],
                         auto_offset_reset='earliest', enable_auto_commit=True,
                         auto_commit_interval_ms=1000, group_id='asm-sql-data-241')

In [ ]:
for message in consumer:
        message_data = json.loads(message.value.decode('utf-8'))
        id_test = message_data['idTest']
        id_paso = message_data['idPaso']
        id_robot = message_data['idRobot']
        fecha_procesado = message_data['fechaProcesado']
        error_sel = message_data['errorSel'].replace("'", '"').replace("]", "")
        element_error = message_data['elementError'].replace("'", '"').replace("]", "")
        print("element_error", element_error)
        datetime_object = datetime.strptime(fecha_procesado, '%Y-%m-%d %H:%M:%S')
    
        # Subtract 2 hours
        modified_datetime = datetime_object - timedelta(hours=2)
        
        # Convert back to the desired format
        fecha_procesado_formateada = modified_datetime.strftime('%Y-%m-%d %H:%M:%S')
        
        print(fecha_procesado_formateada)
        query = f"select * from Resultado WHERE idTest={id_test} AND idRobot={id_robot} AND idPaso={id_paso} AND fecha='{fecha_procesado_formateada}'"
        cursor.execute(query)
        result = cursor.fetchall()
        if (len(result) == 1):
            query_update = f"UPDATE Resultado SET elementError='{element_error}', errorSel='{error_sel}' WHERE idTest={id_test} AND idRobot={id_robot} AND idPaso={id_paso} AND fecha='{fecha_procesado_formateada}'"
            print("query update", query_update)
            cursor.execute(query)
            conn.commit()
        else:
            print("Error: más de una coincidencia o ninguna en base de datos para la query:", query)


In [3]:
# Definir parámetros de conexión
server = "192.168.7.110"  # e.g., "localhost" or "your_server_name\instance_name"
database = "Selenium"
username = "sa"
password = "P@ssw0rd"

# Conexión a la base de datos
conn = pymssql.connect(server=server,user=username,password=password,database=database,as_dict=True,tds_version='7.0')
# Apertura de conexión con SQL
cursor = conn.cursor()

# Execute a sample query
cursor.execute('SELECT TOP 5 * FROM Resultado')  # Replace YourTableName with an actual table name
result = cursor.fetchall()
print("result", result)

result [{'idResultado': 227948406, 'Fecha': datetime.datetime(2021, 9, 1, 0, 0, 3), 'idRobot': 32, 'idTest': 303, 'idPaso': 2, 'idGrupTest': 13295, 'TiempoPaso': 3.571000099182129, 'PathError': None, 'idError': 0, 'errorMsg': None, 'harFile': None, 'urlFailed': None, 'pathPreError': None, 'pathVideo': None, 'idWl': None, 'errorSel': None, 'elementError': None}, {'idResultado': 227948415, 'Fecha': datetime.datetime(2021, 9, 1, 0, 0, 5), 'idRobot': 777, 'idTest': 20008, 'idPaso': 1, 'idGrupTest': 92866, 'TiempoPaso': 0.4339999854564667, 'PathError': None, 'idError': 0, 'errorMsg': None, 'harFile': None, 'urlFailed': None, 'pathPreError': None, 'pathVideo': None, 'idWl': None, 'errorSel': None, 'elementError': None}, {'idResultado': 227948419, 'Fecha': datetime.datetime(2021, 9, 1, 0, 0, 12), 'idRobot': 777, 'idTest': 20006, 'idPaso': 1, 'idGrupTest': 92867, 'TiempoPaso': 0.41200000047683716, 'PathError': None, 'idError': 0, 'errorMsg': None, 'harFile': None, 'urlFailed': None, 'pathPreEr

In [ ]:
for message in consumer:
    try:
        message_data = json.loads(message.value.decode('utf-8'))
        id_test = message_data['idTest']
        id_paso = message_data['idPaso']
        id_robot = message_data['idRobot']
        fecha_procesado = message_data['fecha']
        fecha_procesado = message_data['fecha']
        datetime_object = datetime.strptime(fecha_procesado, '%Y-%m-%d %H:%M:%S')

        # Subtract 2 hours
        modified_datetime = datetime_object - timedelta(hours=2)
        
        # Convert back to the desired format
        fecha_procesado_formateada = modified_datetime.strftime('%Y-%m-%d %H:%M:%S')
        
        print(formatted_date)
        query = f"select * from Resultado WHERE idTest={id_test} AND idRobot={id_robot} AND idPaso={id_paso} AND fecha='{fecha_procesado}'"
        print("query", query)
        cursor.execute(query)
        
        result = cursor.fetchall()
        print("!!!!result", result)
        
    except:
        print("----error", message.timestamp)
        #print("value", message.value)


In [ ]:
consumer.close()
cursor.close()

In [ ]:
!pip install confluent-kafka


In [ ]:
from confluent_kafka import Consumer, KafkaException

conf = {
    'bootstrap.servers': '10.210.4.117:9092,10.210.4.118:9092,10.210.4.119:9092',  # Replace with your Kafka broker's address
    'group.id': 'asm-sql-data-1',
    'auto.offset.reset': 'earliest'
}

consumer = Consumer(conf)

# Subscribe to your Kafka topic
consumer.subscribe(['asm-sql-data'])

In [ ]:


try:
    while True:
        msg = consumer.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaException._PARTITION_EOF:
                continue
            else:
                print(msg.error())
                break

        print('Received message: {}'.format(msg.value().decode('utf-8')))

except KeyboardInterrupt:
    pass

finally:
    consumer.close()